<a href="https://colab.research.google.com/github/MK316/workshops/blob/main/20240531_hufs/240531_HUFS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌌 **Digital Transformation in Classroom: Coding+**

+ Date: May 31, 2024
+ Hosted by: Graduate School of Education at HUFS
+ Online workshop
+ Speaker: Miran Kim (Associate Professor, Dept. of English Education at GNU)

# Objective:

To introduce the basics of Python coding to language teachers, demonstrating how coding can be integrated into language education to enhance digital literacy and teaching methodologies.

Lecture Overview
Introduction (10 minutes)

Welcome and introductions
Brief overview of the lecture's purpose and objectives
Importance of digital transformation in education
Relevance of Python in various educational contexts
Understanding Digital Transformation in Education (15 minutes)

Definition and key concepts of digital transformation
Current trends and technologies in digital education
The role of coding in modern education
Case studies of successful digital integration in classrooms
Introduction to Python Coding (20 minutes)

What is Python and why it is important for educators?
Basic syntax and structure of Python
Hands-on demonstration: Writing your first Python program
Interactive Q&A session to address initial queries
Python for Language Teachers (25 minutes)

How Python can be used in language education
Examples of Python programs for language learning (e.g., vocabulary games, text analysis)
Hands-on activity: Creating a simple vocabulary quiz in Python
Discussion on integrating coding projects into the language curriculum
Practical Applications and Tools (20 minutes)

Overview of educational tools and platforms that support Python coding (e.g., Jupyter Notebooks, Google Colab)
Demonstrating a simple Natural Language Processing (NLP) project for text analysis
Interactive coding session: Basic text analysis with Python
Challenges and Solutions (15 minutes)

Common challenges faced by language teachers in learning and teaching coding
Strategies for overcoming these challenges
Sharing resources and support networks for continuous learning
Conclusion and Q&A (15 minutes)

Recap of key points covered in the lecture
Open floor for questions and discussion
Providing additional resources and references for further learning
Encouraging collaboration and continuous professional development
Suggested Resources
Online Python tutorials and courses (e.g., Codecademy, Coursera)
Python documentation and community forums (e.g., Python.org, Stack Overflow)
Educational tools (e.g., Jupyter Notebooks, Google Colab)
Books and articles on Python in education

# Part I. Introduction

+ Understanding Digital Transformation in Education
+ Introduction to Python Coding

# Part II. Python Coding for Teachers

# Part III. Practical Application and Tools

+ Educational tools and platforms that support Python coding
+ A simple Natural Language Processing (NLP) project for text analysis
+ Interactive coding session

# Part IV. Challenges and Solutions

+ Common challenges faced by language teachers in learning and teaching coding
+ Strategies for overcoming these challenges
+ Sharing resources and support networks for continuous learning

# Q&As

# Suggested Resources

+ Online Python tutorials and courses (e.g., Codecademy, Coursera)
+ Python documentation and community forums (e.g., Python.org, Stack Overflow)
+ Educational tools (e.g., Jupyter Notebooks, Google Colab)
+ Books and articles on Python in education